In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

import pyarrow
import pyarrow.parquet
from sklearn.cluster import KMeans
from shapely.geometry import Point, Polygon
import folium

from folium.plugins import Draw

import matplotlib.pyplot as plt


# 🛰️ AIS Data Filtering and Preprocessing — Kattegat Submarine Cable Area

This notebook documents the initial steps of data preparation for the AIS anomaly detection project focusing on the submarine communication cables in the Kattegat area (Denmark–Sweden).

The goal is to isolate AIS data within a defined polygon surrounding three specific cable routes (**GC2**, **Kattegat 2A**, and **Kattegat 2B**) and perform preliminary cleaning and filtering.


## 🧭 Area of Interest Definition

We are interested in identifying potential anomalous or risky vessel behaviors near submarine cables.

By Danish law, a 200 m protection zone exists on each side of these cables, but in this study we extend the inspection zone to **5 km** to capture behavioral precursors such as early anchoring, trawling, or route deviations before ships enter the restricted corridor.

The polygon defining our area of interest was manually approximated based on the **DKCPC map**.

It includes a section of the Danish coast near Saeby and extends to the Swedish side around Lerkil, covering the main cable routes.  
The coordinates are stored as `(latitude, longitude)` pairs and define an octagonal region enclosing the study zone.


In [2]:
# Define polygon coordinates as (lat, lon)
polygon_coords = [
    (57.3500, 10.5162),  # coast top left
    (57.5120, 10.9314),  # sea top left
    (57.5785, 11.5128),  # sea top right
    (57.5230, 11.9132),  # top right (Swedish coast)
    (57.4078, 11.9189),  # bottom right (Swedish coast)
    (57.1389, 11.2133),  # sea bottom right
    (57.1352, 11.0067),  # sea bottom left
    (57.1880, 10.5400),  # coast bottom left
    (57.3500, 10.5162),  # close polygon (duplicate of first)
]

## ⚙️ Data Loading

In this step, AIS data (in CSV format) is loaded and receives a first hand cleaning

In [ ]:
# df_28 = pd.read_csv("aisdk-2025-10-28.csv")

In [3]:
def mini_cleaning_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    # Remove errors
    bbox = [60, 0, 50, 20]
    north, west, south, east = bbox
    df = df[(df["Latitude"] <= north) & (df["Latitude"] >= south) & (df["Longitude"] >= west) & (
            df["Longitude"] <= east)]

    df = df[df["Type of mobile"].isin(["Class A", "Class B"])].drop(columns=["Type of mobile"])

    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    df = df.drop_duplicates(["Timestamp", "MMSI", ], keep="first")
    df = df.drop(columns=df.columns[-5:])

    knots_to_ms = 0.514444
    df["SOG"] = knots_to_ms * df["SOG"]
    
    return df


In [10]:
def quick_summary(df: pd.DataFrame):
    """
    Generate a quick summary of the AIS data in the DataFrame.
    
    Args:
        df: DataFrame with AIS data, must contain 'MMSI', 'Timestamp', 'Latitude', 'Longitude' columns.
        
    Returns:
        None: Prints summary statistics.
        """
    # Number of unique vessels
    num_vessels = int(df['MMSI'].nunique())
    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")



    # Spatial extent
    lat_min, lat_max = df['Latitude'].min(), df['Latitude'].max()
    lon_min, lon_max = df['Longitude'].min(), df['Longitude'].max()
    centroid_lat, centroid_lon = df['Latitude'].mean(), df['Longitude'].mean()

    # Messages per vessel
    msgs_per_vessel = df.groupby('MMSI').size()
    msgs_stats = msgs_per_vessel.describe().to_dict()
    top_10_vessels = msgs_per_vessel.sort_values(ascending=False).head(10)

    # Print concise overview
    print(f"Number of unique vessels: {num_vessels}")
    print(f"Latitude range: {lat_min:.6f} -- {lat_max:.6f}")
    print(f"Longitude range: {lon_min:.6f} -- {lon_max:.6f}")
    print(f"Centroid (lat, lon): ({centroid_lat:.6f}, {centroid_lon:.6f})")
    print("\nMessages per vessel (summary):")
    for k, v in msgs_stats.items():
        print(f"  {k}: {v}")
    print("\nTop 10 vessels by number of messages (MMSI: count):")
    print(top_10_vessels.to_string())


In [ ]:
df_cleaned = mini_cleaning_pipeline(df_28)

## 📍 Polygon Filtering

The polygon coordinates defined earlier are used to filter the AIS dataset.

Each point is checked to determine whether it falls inside the polygon using geometric operations (e.g. with the `shapely` library).

The resulting dataset contains only positions **within the defined Kattegat zone**, roughly **3–4 %** of the original AIS dataset.


In [ ]:
def is_inside_polygon(lat, lon, polygon_coords):
    """
    Check if a point (lat, lon) is inside a polygon.
    
    Args:
        lat: Latitude
        lon: Longitude
        polygon_coords: List of (lon, lat) tuples defining polygon vertices
        
    Returns:
        Boolean: True if point is inside polygon
    """
    point = Point(lon, lat)
    polygon = Polygon(polygon_coords)
    return polygon.contains(point)

In [ ]:
df_katt = df_cleaned[
    df_cleaned.apply(lambda row: is_inside_polygon(row['Latitude'], row['Longitude'], polygon_coords), axis=1)
]

In [5]:
#here import directly the cleaned one from csv
df_katt = pd.read_csv("df_28_katt.csv")

## 📊 Data Overview After Filtering

We summarize the filtered dataset:

- Number of unique vessels  
- Time coverage of the subset  
- Spatial extent and approximate number of messages per vessel  

This quick overview helps confirm that the filtering worked as expected and that the dataset is representative of the study area.


In [11]:
quick_summary(df_katt)

Number of unique vessels: 172
Latitude range: 57.163097 -- 57.578258
Longitude range: 10.517063 -- 11.867375
Centroid (lat, lon): (57.361891, 11.230398)

Messages per vessel (summary):
  count: 172.0
  mean: 1827.2383720930231
  std: 2845.5646981958407
  min: 2.0
  25%: 544.75
  50%: 633.0
  75%: 1068.25
  max: 17475.0

Top 10 vessels by number of messages (MMSI: count):
MMSI
219009229    17475
220279000    11712
220323000    10512
220046000     9308
219010207     9083
219006219     8854
220334000     8836
219001553     8814
219000543     8698
220349000     8662


## 🗺️ Quick Visualization with Folium

To get an intuitive sense of the vessel trajectories, we plot the positions of a single ship (selected by its `MMSI`) on an interactive map using **Folium**.

This allows us to quickly verify that:
- The AIS points are correctly located within the Kattegat inspection area.  
- The temporal resampling (e.g., every 5 minutes) still preserves the vessel’s overall path.  
- The filtering and cleaning steps did not introduce spatial artifacts.

Each marker on the map corresponds to one AIS position for the selected vessel at a given timestamp.


In [ ]:
df_nave1 = df_katt[df_katt["MMSI"] == 219009229] #select mmsi

# Ensure 'Timestamp' is datetime and sort chronologically
df_nave1['Timestamp'] = pd.to_datetime(df_nave1['Timestamp'], errors='coerce')
df_nave1 = df_nave1.sort_values('Timestamp')

# Resample every 5 minutes (use '5min' instead of deprecated '5T')
df_nave1_5min = (
    df_nave1
    .set_index('Timestamp')
    .resample('65min')
    .first()              # take first observation in each 10-min bin
    .dropna(how='all')
    .reset_index()
)

df_nave1_5min.shape


In [ ]:
# Create map centered on Kattegat/Læsø
map_center = [57.35, 11.0]  # [lat, lon]
m = folium.Map(
    location=map_center,
    zoom_start=9,
    tiles='OpenStreetMap'
)

coordinates = list(zip(df_nave1_5min['Longitude'], df_nave1_5min['Latitude']))

# Add vertex markers as triangles
for i, (lon, lat) in enumerate(coordinates[:-1]):  # Skip last (duplicate)
    folium.Marker(
        location=[lat, lon],
        popup=f'Vertex {i+1}<br>Lat: {lat:.4f}<br>Lon: {lon:.4f}',
        tooltip=f'V{i+1}',
        icon=folium.Icon(
            icon='f2ec',
            prefix='fa',  # Font Awesome icons
            color='red',
            icon_color='white'
        )
    ).add_to(m)
